<a href="https://colab.research.google.com/github/Tommy-Las/WatfordFC/blob/main/Data_cleaning_transformation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# If google colab

In [ ]:
# Mount drive

from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

# Import packages and data

In [2]:
import pandas as pd
# Turn off warnings
pd.options.mode.chained_assignment = None
pd.set_option('display.max_columns', None)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import KNNImputer
from sklearn.preprocessing import OneHotEncoder

Import GPS data

In [3]:
# file_path = '/content/drive/MyDrive/WATFORD FC/Datos GPS/GPS 2018-2023.xlsx'
file_path = 'data/GPS 2018-2023_NoContact.xlsx'
df_gps = pd.read_excel(file_path)

Import speed data

In [4]:
# file_path = '/content/drive/MyDrive/WATFORD FC/Datos GPS/max_speed.xlsx'
file_path = 'data/max_speed.xlsx'
df_speed = pd.read_excel(file_path)

Import wellbeing data


In [5]:
#file_path = '/content/drive/MyDrive/WATFORD FC/Datos Wellbeing/wellbeing ssp.xlsx'
#df_wellbeing = pd.read_excel(file_path)

# Merge Sprint rows into a single row per session

The sprint values are separated in different rows per session ,

We want to merge in into a single row per session.

## Transform data before merging sprint rows

In [6]:
df_speed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123180 entries, 0 to 123179
Data columns (total 11 columns):
 #   Column                                       Non-Null Count   Dtype  
---  ------                                       --------------   -----  
 0   DATE                                         123180 non-null  object 
 1   ID                                           123180 non-null  int64  
 2   Player Position                              123155 non-null  object 
 3   Max Speed                                    116475 non-null  float64
 4   Sprints                                      113560 non-null  float64
 5   MINUTES                                      113654 non-null  float64
 6   Season                                       123180 non-null  object 
 7   Max Speed Season                             123180 non-null  float64
 8   Avg Speed Season                             122357 non-null  float64
 9   % Max Speed                                  122357 non-nul

We want to remove rows after 08-07-2021 since we don't have data for sprints and speed

In [7]:
from datetime import date
# Convert DATE column values into Pandas datetime object
df_speed['DATE'] = pd.to_datetime(df_speed['DATE'], dayfirst=True)

# Filter rows after date: '2021-08-07'
date_filter = pd.Timestamp('2021-08-07')
df_speed = df_speed[df_speed['DATE'] > date_filter]

# Filter rows after date: '2023-05-11'
date_filter = pd.Timestamp('2023-05-11')
df_speed = df_speed[df_speed['DATE'] < date_filter]

date_filter = pd.Timestamp('2021-06-28')
df_gps = df_gps[df_gps['DATE'] > date_filter]

In [ ]:
df_speed.info()

<class 'pandas.core.frame.DataFrame'>
Index: 55390 entries, 58257 to 113646
Data columns (total 11 columns):
 #   Column                                       Non-Null Count  Dtype         
---  ------                                       --------------  -----         
 0   DATE                                         55390 non-null  datetime64[ns]
 1   ID                                           55390 non-null  int64         
 2   Player Position                              55365 non-null  object        
 3   Max Speed                                    55377 non-null  float64       
 4   Sprints                                      55362 non-null  float64       
 5   MINUTES                                      55389 non-null  float64       
 6   Season                                       55390 non-null  object        
 7   Max Speed Season                             55390 non-null  float64       
 8   Avg Speed Season                             55390 non-null  float64       


Replace NULL sprint values to 0

In [9]:
# Replace NULL values from Sprint to 0
df_speed['Sprints'] = df_speed['Sprints'].fillna(0)

## Merge sprint values

Print values to compare merge later

In [10]:
df_speed.sort_values(by='DATE', ascending=False).head(15)

,DATE,ID,Player Position,Max Speed,Sprints,MINUTES,Season,Max Speed Season,Avg Speed Season,% Max Speed,%Speed diference against max. Speed average
58257,2023-05-10,23085,CENTRE MIDFIELDER,22.91,3.0,30.19,2022-2023,32.12,20.304388,71.326276,12.832753
58258,2023-05-10,87583,CENTRE MIDFIELDER,24.70,20.0,40.53,2022-2023,34.27,21.189939,72.074701,16.564751
58259,2023-05-09,96811,CENTRE MIDFIELDER,32.19,10.0,70.51,2022-2023,36.04,20.882454,89.317425,54.148548
58260,2023-05-09,89871,STRIKER,25.46,0.0,79.87,2022-2023,34.12,21.753402,74.618992,17.039162
58300,2023-05-08,55555,FULL BACK,30.47,18.0,72.62,2022-2023,34.94,22.071250,87.206640,38.052897
58289,2023-05-08,48692,CENTRE BACK,32.56,35.0,69.15,2022-2023,36.21,21.029270,89.919912,54.831811
58290,2023-05-08,87583,CENTRE MIDFIELDER,22.69,11.0,34.13,2022-2023,34.27,21.189939,66.209513,7.079117
58291,2023-05-08,37537,CENTRE MIDFIELDER,28.65,5.0,47.05,2022-2023,34.09,21.974456,84.042241,30.378655
58292,2023-05-08,37537,CENTRE MIDFIELDER,30.59,3.0,22.10,2022-2023,34.09,21.974456,89.733060,39.207088
58293,2023-05-08,37537,CENTRE MIDFIELDER,30.59,8.0,69.15,2022-2023,34.09,21.974456,89.733060,39.207088


Do a group by player and date, and select the maximum value for each feature

In [11]:
df_speed = df_speed.groupby(['DATE', 'ID']).agg('max').reset_index()

In [ ]:
df_speed.sort_values(by='DATE', ascending=False).head(15)

,DATE,ID,Player Position,Max Speed,Sprints,MINUTES,Season,Max Speed Season,Avg Speed Season,% Max Speed,%Speed diference against max. Speed average
8262,2023-05-10,87583,CENTRE MIDFIELDER,24.70,20.0,40.53,2022-2023,34.27,21.189939,72.074701,16.564751
8261,2023-05-10,23085,CENTRE MIDFIELDER,22.91,3.0,30.19,2022-2023,32.12,20.304388,71.326276,12.832753
8260,2023-05-09,96811,CENTRE MIDFIELDER,32.19,10.0,70.51,2022-2023,36.04,20.882454,89.317425,54.148548
8259,2023-05-09,89871,STRIKER,25.46,0.0,79.87,2022-2023,34.12,21.753402,74.618992,17.039162
8249,2023-05-08,55555,FULL BACK,30.47,18.0,72.62,2022-2023,34.94,22.071250,87.206640,38.052897
8240,2023-05-08,10103,FULL BACK,21.24,1.0,44.84,2022-2023,34.95,22.202230,60.772532,-4.333932
8241,2023-05-08,12086,WINGER,35.04,41.0,88.27,2022-2023,35.44,21.901184,98.871332,59.991349
8242,2023-05-08,19817,FULL BACK,31.68,0.0,128.04,2022-2023,35.77,21.605460,88.565837,46.629599
8243,2023-05-08,21079,WINGER,34.51,51.0,88.04,2022-2023,36.97,22.708654,93.345956,51.968499
8244,2023-05-08,23085,CENTRE MIDFIELDER,23.71,7.0,58.52,2022-2023,32.12,20.304388,73.816936,16.772788


# Handle duplicates for GPS Data

There are some players that have 2 records for the same day, they are different amounts so we sum them

In [13]:
# Define columns to sum
columns_to_sum = ['Total D', '>19.8', '> 25 Km/h', 'ACC', 'DEC']

# Define columns to select the first value
columns_to_first = ['DATE', 'Column2', 'PLAYER', 'Injury', 'season', 'LEAGUE', 'preseason-season', 'MANAGER']

# Group by the duplicate subset and aggregate
df_gps_aggregated = (
    df_gps[df_gps.duplicated(subset=['PLAYER', 'DATE'], keep=False)]
    .groupby(['PLAYER', 'DATE'], as_index=False)
    .agg({**{col: 'sum' for col in columns_to_sum},
          **{col: 'first' for col in columns_to_first}})
)

# Ensure non-duplicated rows are preserved by combining them back
df_gps_combined = pd.concat([
    df_gps[~df_gps.duplicated(subset=['PLAYER', 'DATE'], keep=False)],
    df_gps_aggregated
], ignore_index=True)

# Merge GPS and Speed dataframes

## Prepare the data before the merge

Verify column names for both Dataframes

In [14]:
df_gps_combined.columns

Index(['DATE', 'Column2', 'PLAYER', 'Injury', 'season', 'LEAGUE',
       'preseason-season', 'MANAGER', 'Total D', '>19.8', '> 25 Km/h', 'ACC',
       'DEC'],
      dtype='object')

In [15]:
df_speed.columns

Index(['DATE', 'ID', 'Player Position', 'Max Speed', 'Sprints', 'MINUTES',
       'Season', 'Max Speed Season', 'Avg Speed Season', '% Max Speed',
       '%Speed diference against max. Speed average'],
      dtype='object')

Change datatypes before the merge

In [16]:
# Convert DATE column from GPS df into datetime pandas obkect
df_gps_combined['DATE'] = pd.to_datetime(df_gps_combined['DATE'], dayfirst=True)

# Drop NULL values for 'PLAYER'
df_gps_combined = df_gps_combined.dropna(subset=['PLAYER'])

# Convert ID and PLAYER columns to the same data type - integers
df_gps_combined['PLAYER'] = df_gps_combined['PLAYER'].astype(int)
df_speed['ID'] = df_speed['ID'].astype(int)

## Do the merge of gps & speed dataframes

In [17]:
# Perform an inner join on matching DATE and PLAYER/ID values
df_merged = df_gps_combined.merge(df_speed, left_on=['DATE', 'PLAYER'], right_on=['DATE', 'ID'], how='inner')

In [ ]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7835 entries, 0 to 7834
Data columns (total 23 columns):
 #   Column                                       Non-Null Count  Dtype         
---  ------                                       --------------  -----         
 0   DATE                                         7835 non-null   datetime64[ns]
 1   Column2                                      7835 non-null   object        
 2   PLAYER                                       7835 non-null   int64         
 3   Injury                                       7835 non-null   float64       
 4   season                                       7835 non-null   object        
 5   LEAGUE                                       7835 non-null   object        
 6   preseason-season                             7835 non-null   object        
 7   MANAGER                                      7835 non-null   object        
 8   Total D                                      7835 non-null   object        
 9

Verify class imbalances

In [ ]:
print("No Injury: " + str(df_merged[df_merged["Injury"] == 0].shape[0]))
print("Injury: " + str(df_merged[df_merged["Injury"] == 1].shape[0]))
print(f"%: {((df_merged[df_merged['Injury'] == 1].shape[0] / df_merged.shape[0]) * 100):.2f}%")

No Injury: 7798
Injury: 37
%: 0.47%


# Merge df with 'weight' in wellbeing dataframe

In [20]:
# df_wellbeing = df_wellbeing[["Weight", "Date", "PLAYER"]]
# df_wellbeing.info()

In [21]:
# # Sort by PLAYER and DATE to ensure correct order for backfilling
# df_wellbeing = df_wellbeing.sort_values(by=['PLAYER', 'Date'])

# # Fill the NULL values in 'Weight' with the previous valid value for each player
# df_wellbeing['Weight'] = df_wellbeing.groupby('PLAYER')['Weight'].bfill()
# df_wellbeing['Weight'] = df_wellbeing.groupby('PLAYER')['Weight'].ffill()

# df_wellbeing.info()

In [22]:
# # Merge the DataFrames
# df_merged = df_merged.merge(df_wellbeing, left_on=['DATE', 'PLAYER'], right_on=['Date', 'PLAYER'], how='left')

# # Sort by PLAYER and DATE to ensure chronological order
# df_merged = df_merged.sort_values(by=['PLAYER', 'DATE'])

# # Back-fill null values in 'Weight' for each PLAYER
# df_merged['Weight'] = df_merged.groupby('PLAYER')['Weight'].bfill()
# df_merged['Weight'] = df_merged.groupby('PLAYER')['Weight'].ffill()

# # Optional: Drop the 'Date' column from the right DataFrame if not needed
# df_merged = df_merged.drop(columns=['Date'])

# # Display the final DataFrame
# df_merged.info()

In [23]:
#df_merged.groupby('PLAYER')['Weight'].apply(lambda x: x.isnull().sum())

# Transform numeric columns to the same data types

In [24]:
cols = ['Total D', '>19.8', '> 25 Km/h', 'ACC',
       'DEC', 'ID', 'Max Speed', 'Sprints', 'MINUTES', 'Max Speed Season',
       'Avg Speed Season', '% Max Speed',
       '%Speed diference against max. Speed average']

df_merged[cols] = df_merged[cols].astype(float)

In [ ]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7835 entries, 0 to 7834
Data columns (total 23 columns):
 #   Column                                       Non-Null Count  Dtype         
---  ------                                       --------------  -----         
 0   DATE                                         7835 non-null   datetime64[ns]
 1   Column2                                      7835 non-null   object        
 2   PLAYER                                       7835 non-null   int64         
 3   Injury                                       7835 non-null   float64       
 4   season                                       7835 non-null   object        
 5   LEAGUE                                       7835 non-null   object        
 6   preseason-season                             7835 non-null   object        
 7   MANAGER                                      7835 non-null   object        
 8   Total D                                      7835 non-null   float64       
 9

# Values that are 0 for ACC, DEC, Max Speed

Perform imputations on missing data

In [26]:
# Count of rows containing 0 for each column
zero_counts = (df_merged == 0).sum()

# Creating a DataFrame for the results
result_df = pd.DataFrame({
    "Column Name": zero_counts.index,
    "Number of Rows with 0": zero_counts.values
})

result_df

,Column Name,Number of Rows with 0
0,DATE,0
1,Column2,382
2,PLAYER,0
3,Injury,7798
4,season,0
5,LEAGUE,0
6,preseason-season,0
7,MANAGER,0
8,Total D,3
9,>19.8,420


Perform KNN Imputation

In [27]:
# Columns to impute
metrics_imputation_zeroes = ['Total D', 'ACC', 'DEC', 'Max Speed', 'MINUTES']

# Columns to use for KNN calculation
columns_for_knn = ['Total D', '>19.8', '> 25 Km/h', 'ACC', 'DEC', 'ID',
                   'Max Speed', 'Sprints', 'MINUTES', '% Max Speed']

# Replace 0 with NaN in the columns to be imputed
df_merged[metrics_imputation_zeroes] = df_merged[metrics_imputation_zeroes].replace(0, np.nan)

# Ensure the columns used for KNN calculation are numeric
columns_for_knn_numeric = df_merged[columns_for_knn].select_dtypes(include=[np.number]).columns.tolist()

# Initialize the KNNImputer
knn_imputer = KNNImputer(n_neighbors=5)  # Adjust n_neighbors if needed

# Perform KNN imputation using the broader set of columns for calculation
imputed_values = knn_imputer.fit_transform(df_merged[columns_for_knn_numeric])

# Update only the specified columns to impute
df_merged[metrics_imputation_zeroes] = imputed_values[:,
    [columns_for_knn_numeric.index(col) for col in metrics_imputation_zeroes]]

In [ ]:
# Count of rows containing 0 for each column
zero_counts = (df_merged == 0).sum()

# Creating a DataFrame for the results
result_df = pd.DataFrame({
    "Column Name": zero_counts.index,
    "Number of Rows with 0": zero_counts.values
})

result_df

,Column Name,Number of Rows with 0
0,DATE,0
1,Column2,382
2,PLAYER,0
3,Injury,7798
4,season,0
5,LEAGUE,0
6,preseason-season,0
7,MANAGER,0
8,Total D,0
9,>19.8,420


Calculate '% Max Speed' and '% Speed difference against max. Speed average'

In [29]:
# Calculate '% Max Speed' for all rows
df_merged['% Max Speed'] = (df_merged['Max Speed'] / df_merged['Max Speed Season']) * 100

# Calculate '% Speed difference against max. Speed average' for all rows
df_merged['%Speed diference against max. Speed average'] = (
    (df_merged['Max Speed'] - df_merged['Avg Speed Season']) / df_merged['Avg Speed Season']
) * 100

# Reduce data volume/size

## Players who have never been injured

We are going to filter by players that had at least one injury, to reduce # of rows and hopefully improve accuracy

In [30]:
# Group by ID and sum the Injury column
injury_counts = df_merged.groupby('ID').agg({'Injury': 'sum'})

# Get the IDs where the sum of Injury is greater than 0
ids_to_remove = injury_counts[injury_counts['Injury'] == 0].index.tolist()

# Remove these IDs from the original DataFrame
df_filtered = df_merged[~df_merged['ID'].isin(ids_to_remove)]

In [31]:
print("No Injury: " + str(df_filtered[df_filtered["Injury"] == 0].shape[0]))
print("Injury: " + str(df_filtered[df_filtered["Injury"] == 1].shape[0]))
print(f"%: {((df_filtered[df_filtered['Injury'] == 1].shape[0] / df_filtered.shape[0]) * 100):.2f}%")

No Injury: 4686
Injury: 37
%: 0.78%


Verify that at least one player had an injury

In [ ]:
df_filtered.groupby('ID').agg({'Injury': 'sum'})

,Injury
ID,
10103.0,4.0
10452.0,2.0
12086.0,1.0
17316.0,2.0
18096.0,3.0
20083.0,1.0
21079.0,2.0
23081.0,2.0
23085.0,2.0


# Change column names & drop unnecesary columns

In [ ]:
column_rename_dict = {
    'Column2': 'Microcycle',
    'DATE': 'DATE',
    'ID': 'PlayerID',
    'Total D': 'TD',
    '>19.8': '>19.8',
    '> 25 Km/h': '>25',
    'ACC': 'ACC',
    'DEC': 'DEC',
    'Max Speed': 'Max Speed',
    'Max Speed Season': 'Max Speed Season',
    'Avg Speed Season': 'Avg Speed Season',
    '% Max Speed': '% Max Speed',
    '%Speed diference against max. Speed average': 'Speed Diff Max Avg',
    'Injury': 'Injury',
    'MINUTES': 'Mins',
    'Sprints': 'Sprints',
    'Total D_Rel': 'TD_Rel',
    '>19.8_Rel': '>19.8_Rel',
    '> 25 Km/h_Rel': '>25 Km/h_Rel',
    'ACC_Rel': 'ACC_Rel',
    'DEC_Rel': 'DEC_Rel',
    'Sprints_Rel': 'Sprints_Rel'
}

df_filtered.rename(columns=column_rename_dict, inplace=True)

df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4723 entries, 0 to 7802
Data columns (total 23 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   DATE                4723 non-null   datetime64[ns]
 1   Microcycle          4723 non-null   object        
 2   PLAYER              4723 non-null   int64         
 3   Injury              4723 non-null   float64       
 4   season              4723 non-null   object        
 5   LEAGUE              4723 non-null   object        
 6   preseason-season    4723 non-null   object        
 7   MANAGER             4723 non-null   object        
 8   TD                  4723 non-null   float64       
 9   >19.8               4723 non-null   float64       
 10  >25                 4723 non-null   float64       
 11  ACC                 4723 non-null   float64       
 12  DEC                 4723 non-null   float64       
 13  PlayerID            4723 non-null   float64       
 1

In [34]:
df_filtered = df_filtered.drop(columns=['LEAGUE', 'MANAGER', 'PLAYER', 'Season'])

# Calculate relative values

Remove spaces in 'Microcycle' column

In [35]:
df_filtered['Microcycle'] = df_filtered['Microcycle'].str.replace(' ', '', regex=False)
df_filtered["Microcycle"].unique()

array(['M+2', 'M+1', 'MD', 'M-1', 'M-2', 'M-3', 'M-5', 'M+3', 'M-4',
       'MD(AWAY)', 'MD(HOME)', nan], dtype=object)

To calculate the relative values, we want the max and avg values for players that played over 85 in a Match Day

In [36]:
def calculate_relative_values(df_original, metrics):
    # Array that holds match days
    microcycle_values = ['MD', 'MD(HOME)', 'MD(AWAY)']

    # Do a copy of the original DF
    df_copy = df_original.copy()

    # Filter players with +85 mins and match day
    df_filtered_85_md = df_copy[
        (df_copy['Mins'] > 85) &
        (df_copy['Microcycle'].isin(microcycle_values))
    ]

    # Players who never completed 85 mins in matchday
    df_filtered_85_only_mins = df_copy[df_copy['Mins'] > 85]

    # Combine both DataFrames
    df_filtered_85 = pd.concat([df_filtered_85_md, df_filtered_85_only_mins])

    # Temporary dictionaries to hold max and avg values per player
    player_max = {}
    player_avg = {}

    # Calculate max and avg for each player
    for player in df_filtered_85['PlayerID'].unique():
        player_data = df_filtered_85[df_filtered_85['PlayerID'] == player]
        player_max[player] = player_data[metrics].max()
        player_avg[player] = player_data[metrics].mean()

    # Add relative values to the original DataFrame
    for metric in metrics:

        df_original[f"{metric}_Rel"] = np.nan

        for player in df_original['PlayerID'].unique():
            if player in player_max and player in player_avg:
                max_value = player_max[player][metric]
                avg_value = player_avg[player][metric]

                # Compute relative value
                df_original.loc[df_original['PlayerID'] == player, f"{metric}_Rel"] = (
                    (df_original.loc[df_original['PlayerID'] == player, metric] * 100) /
                    ((max_value + avg_value) / 2)
                ).round(2)

    return df_original

Call function to calculate relative values

In [37]:
metrics_rel = ['TD', '>19.8', '>25', 'ACC', 'DEC']

# Avoid for now
df_rel = calculate_relative_values(df_filtered, metrics_rel)

# df_rel = df_filtered.copy()

# Function for densities

In [38]:
# cols_mins = ['TD', 'HSR', '+25 Km/h', 'ACC', 'DEC', 'Sprints']

# # Create new columns by dividing by the 'Minutes' column
# for col in cols_mins:
#     df_rel[f'{col}/Mins'] = df_rel[col] / df_rel['Mins']

In [39]:
# df_rel['Sprints/Mins'] = df_rel['Sprints/Mins'].fillna(0)

# Player loads

## Calculate loads

In [40]:
def calcular_acumulado(df, columnas_calcular, dias):
    # Columns to exclude when the day is 3
    excluded_columns_3_days = ['TD_Rel', '>19.8_Rel', '>25_Rel', 'ACC_Rel', 'DEC_Rel', '% Max Speed']

    # Create an empty list to store processed player DataFrames
    processed_players = []

    # Process each player separately
    for player_id in df['PlayerID'].unique():
        # Filter data for the current player
        player_data = df[df['PlayerID'] == player_id].copy()

        # Create a full date range for the player (from the first to the last recorded date)
        full_date_range = pd.date_range(start=player_data['DATE'].min(), end=player_data['DATE'].max(), freq='D')

        # Set 'DATE' as the index and reindex to fill missing dates with zeros
        player_data = player_data.set_index('DATE').reindex(full_date_range, fill_value=0).reset_index()
        player_data.rename(columns={'index': 'DATE'}, inplace=True)
        player_data['PlayerID'] = player_id

        # Perform rolling calculations for each metric, excluding the current day
        for dia in dias:
            for col in columnas_calcular:
                # Skip excluded columns when the day is 3
                if dia != 1 and col in excluded_columns_3_days:
                    continue

                # Check if the column exists to avoid errors
                if col in player_data.columns:
                    if dia in [7, 21]:
                        # Rolling sum, mean, and std for 7 and 21 days
                        player_data[f'{col}-{dia}'] = (
                            player_data[col].shift(1).rolling(window=dia, min_periods=1).sum()
                        )
                        player_data[f'{col}-{dia}-avg'] = (
                            player_data[col].shift(1).rolling(window=dia, min_periods=1).mean()
                        )
                        player_data[f'{col}-{dia}-std'] = (
                            player_data[col].shift(1).rolling(window=dia, min_periods=1).std()
                        )
                    else:
                        # Only rolling sum for other periods
                        player_data[f'{col}-{dia}'] = (
                            player_data[col].shift(1).rolling(window=dia, min_periods=1).sum()
                        )

        # Drop rows where all calculated values are zero (rest days)
        mask_non_zero = (player_data[columnas_calcular].sum(axis=1) > 0)
        player_data = player_data[mask_non_zero]

        # Append the processed player's data to the list
        processed_players.append(player_data)

    # Concatenate all processed player DataFrames into a single DataFrame
    df_resultado = pd.concat(processed_players, ignore_index=True)

    return df_resultado


In [41]:
cols_calculate = ['TD', '>19.8', '>25', 'ACC', 'DEC', 'Sprints', 'Mins', 'TD_Rel', '>19.8_Rel', '>25_Rel',
       'ACC_Rel', 'DEC_Rel', '% Max Speed']

cumulative_df = calcular_acumulado(df_rel, cols_calculate, [1,3,7,21]) # loads -1, -3, -7, -21

In [42]:
cumulative_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4723 entries, 0 to 4722
Data columns (total 86 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   DATE                4723 non-null   datetime64[ns]
 1   Microcycle          4508 non-null   object        
 2   Injury              4723 non-null   float64       
 3   season              4723 non-null   object        
 4   preseason-season    4723 non-null   object        
 5   TD                  4723 non-null   float64       
 6   >19.8               4723 non-null   float64       
 7   >25                 4723 non-null   float64       
 8   ACC                 4723 non-null   float64       
 9   DEC                 4723 non-null   float64       
 10  PlayerID            4723 non-null   float64       
 11  Player Position     4723 non-null   object        
 12  Max Speed           4723 non-null   float64       
 13  Sprints             4723 non-null   float64     

In [43]:
cumulative_df.head()

,DATE,Microcycle,Injury,season,preseason-season,TD,>19.8,>25,ACC,DEC,PlayerID,Player Position,Max Speed,Sprints,Mins,Max Speed Season,Avg Speed Season,% Max Speed,Speed Diff Max Avg,TD_Rel,>19.8_Rel,>25_Rel,ACC_Rel,DEC_Rel,TD-1,>19.8-1,>25-1,ACC-1,DEC-1,Sprints-1,Mins-1,TD_Rel-1,>19.8_Rel-1,>25_Rel-1,ACC_Rel-1,DEC_Rel-1,% Max Speed-1,TD-3,>19.8-3,>25-3,ACC-3,DEC-3,Sprints-3,Mins-3,TD-7,TD-7-avg,TD-7-std,>19.8-7,>19.8-7-avg,>19.8-7-std,>25-7,>25-7-avg,>25-7-std,ACC-7,ACC-7-avg,ACC-7-std,DEC-7,DEC-7-avg,DEC-7-std,Sprints-7,Sprints-7-avg,Sprints-7-std,Mins-7,Mins-7-avg,Mins-7-std,TD-21,TD-21-avg,TD-21-std,>19.8-21,>19.8-21-avg,>19.8-21-std,>25-21,>25-21-avg,>25-21-std,ACC-21,ACC-21-avg,ACC-21-std,DEC-21,DEC-21-avg,DEC-21-std,Sprints-21,Sprints-21-avg,Sprints-21-std,Mins-21,Mins-21-avg,Mins-21-std
0,2021-08-10,M-4,0.0,2021-2022,SEASON,5651.0,197.0,34.0,63.0,36.0,23085.0,CENTRE MIDFIELDER,28.41,19.0,80.35,30.73,20.278653,92.450374,40.098060,52.19,34.13,44.42,77.81,41.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-08-11,M-3,0.0,2021-2022,SEASON,7423.0,426.0,20.0,52.0,38.0,23085.0,CENTRE MIDFIELDER,26.79,24.0,90.85,30.73,20.278653,87.178653,32.109364,68.55,73.81,26.13,64.23,43.80,5651.0,197.0,34.0,63.0,36.0,19.0,80.35,52.19,34.13,44.42,77.81,41.50,92.450374,5651.0,197.0,34.0,63.0,36.0,19.0,80.35,5651.0,5651.000000,NaN,197.0,197.000000,NaN,34.0,34.0,NaN,63.0,63.0,NaN,36.0,36.000000,NaN,19.0,19.000000,NaN,80.35,80.350000,NaN,5651.0,5651.000000,NaN,197.0,197.000000,NaN,34.0,34.0,NaN,63.0,63.0,NaN,36.0,36.000000,NaN,19.0,19.000000,NaN,80.35,80.350000,NaN
2,2021-08-12,M-2,0.0,2021-2022,SEASON,1268.0,0.0,0.0,11.0,5.0,23085.0,CENTRE MIDFIELDER,16.57,0.0,43.58,30.73,20.278653,53.921250,-18.288460,11.71,0.00,0.00,13.59,5.76,7423.0,426.0,20.0,52.0,38.0,24.0,90.85,68.55,73.81,26.13,64.23,43.80,87.178653,13074.0,623.0,54.0,115.0,74.0,43.0,171.20,13074.0,6537.000000,1252.993216,623.0,311.500000,161.927453,54.0,27.0,9.899495,115.0,57.5,7.778175,74.0,37.000000,1.414214,43.0,21.500000,3.535534,171.20,85.600000,7.424621,13074.0,6537.000000,1252.993216,623.0,311.500000,161.927453,54.0,27.0,9.899495,115.0,57.5,7.778175,74.0,37.000000,1.414214,43.0,21.500000,3.535534,171.20,85.600000,7.424621
3,2021-08-13,M-1,0.0,2021-2022,SEASON,4135.0,112.0,2.0,47.0,20.0,23085.0,CENTRE MIDFIELDER,25.47,11.0,58.35,30.73,20.278653,82.883176,25.600056,38.19,19.40,2.61,58.05,23.05,1268.0,0.0,0.0,11.0,5.0,0.0,43.58,11.71,0.00,0.00,13.59,5.76,53.921250,14342.0,623.0,54.0,126.0,79.0,43.0,214.78,14342.0,4780.666667,3168.456459,623.0,207.666667,213.200219,54.0,18.0,17.088007,126.0,42.0,27.404379,79.0,26.333333,18.502252,43.0,14.333333,12.662280,214.78,71.593333,24.821818,14342.0,4780.666667,3168.456459,623.0,207.666667,213.200219,54.0,18.0,17.088007,126.0,42.0,27.404379,79.0,26.333333,18.502252,43.0,14.333333,12.662280,214.78,71.593333,24.821818
4,2021-08-15,M+1,0.0,2021-2022,SEASON,6379.0,86.0,0.0,49.0,48.0,23085.0,CENTRE MIDFIELDER,23.37,13.0,79.48,30.73,20.278653,76.049463,15.244339,58.91,14.90,0.00,60.52,55.33,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,5403.0,112.0,2.0,58.0,25.0,11.0,101.93,18477.0,3695.400000,3060.264417,735.0,147.000000,176.623894,56.0,11.2,15.270887,173.0,34.6,27.464523,99.0,19.800000,17.355115,54.0,10.800000,10.894953,273.13,54.626000,35.685377,18477.0,3695.400000,3060.264417,735.0,147.000000,176.623894,56.0,11.2,15.270887,173.0,34.6,27.464523,99.0,19.800000,17.355115,54.0,10.800000,10.894953,273.13,54.626000,35.685377


## For player loads, drop rows with NULL or 0 values

In [44]:
cumulative_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4723 entries, 0 to 4722
Data columns (total 86 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   DATE                4723 non-null   datetime64[ns]
 1   Microcycle          4508 non-null   object        
 2   Injury              4723 non-null   float64       
 3   season              4723 non-null   object        
 4   preseason-season    4723 non-null   object        
 5   TD                  4723 non-null   float64       
 6   >19.8               4723 non-null   float64       
 7   >25                 4723 non-null   float64       
 8   ACC                 4723 non-null   float64       
 9   DEC                 4723 non-null   float64       
 10  PlayerID            4723 non-null   float64       
 11  Player Position     4723 non-null   object        
 12  Max Speed           4723 non-null   float64       
 13  Sprints             4723 non-null   float64     

Drop rows that contain more than 2 NULL values

In [45]:
# Exclude 'Microcycle' column for counting NULLs
columns_to_check = cumulative_df.drop(columns=['Microcycle']).columns

# Drop rows where 2 or more null values exist (excluding 'Microcycle')
cumulative_df = cumulative_df[cumulative_df[columns_to_check].notna().sum(axis=1) > (len(columns_to_check) - 1)]

# Display the resulting DataFrame
cumulative_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4680 entries, 2 to 4722
Data columns (total 86 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   DATE                4680 non-null   datetime64[ns]
 1   Microcycle          4465 non-null   object        
 2   Injury              4680 non-null   float64       
 3   season              4680 non-null   object        
 4   preseason-season    4680 non-null   object        
 5   TD                  4680 non-null   float64       
 6   >19.8               4680 non-null   float64       
 7   >25                 4680 non-null   float64       
 8   ACC                 4680 non-null   float64       
 9   DEC                 4680 non-null   float64       
 10  PlayerID            4680 non-null   float64       
 11  Player Position     4680 non-null   object        
 12  Max Speed           4680 non-null   float64       
 13  Sprints             4680 non-null   float64       
 1

In [46]:
# Count of rows containing 0 for each column
zero_counts = (cumulative_df == 0).sum()

# Creating a DataFrame for the results
result_df = pd.DataFrame({
    "Column Name": zero_counts.index,
    "Number of Rows with 0": zero_counts.values
})

result_df

,Column Name,Number of Rows with 0
0,DATE,0
1,Microcycle,0
2,Injury,4643
3,season,0
4,preseason-season,0
...,...,...
81,Sprints-21-avg,157
82,Sprints-21-std,157
83,Mins-21,46
84,Mins-21-avg,46


Remove rows where 3 or more rows contain 0 for the loads

In [47]:
cumulative_df.columns

Index(['DATE', 'Microcycle', 'Injury', 'season', 'preseason-season', 'TD',
       '>19.8', '>25', 'ACC', 'DEC', 'PlayerID', 'Player Position',
       'Max Speed', 'Sprints', 'Mins', 'Max Speed Season', 'Avg Speed Season',
       '% Max Speed', 'Speed Diff Max Avg', 'TD_Rel', '>19.8_Rel', '>25_Rel',
       'ACC_Rel', 'DEC_Rel', 'TD-1', '>19.8-1', '>25-1', 'ACC-1', 'DEC-1',
       'Sprints-1', 'Mins-1', 'TD_Rel-1', '>19.8_Rel-1', '>25_Rel-1',
       'ACC_Rel-1', 'DEC_Rel-1', '% Max Speed-1', 'TD-3', '>19.8-3', '>25-3',
       'ACC-3', 'DEC-3', 'Sprints-3', 'Mins-3', 'TD-7', 'TD-7-avg', 'TD-7-std',
       '>19.8-7', '>19.8-7-avg', '>19.8-7-std', '>25-7', '>25-7-avg',
       '>25-7-std', 'ACC-7', 'ACC-7-avg', 'ACC-7-std', 'DEC-7', 'DEC-7-avg',
       'DEC-7-std', 'Sprints-7', 'Sprints-7-avg', 'Sprints-7-std', 'Mins-7',
       'Mins-7-avg', 'Mins-7-std', 'TD-21', 'TD-21-avg', 'TD-21-std',
       '>19.8-21', '>19.8-21-avg', '>19.8-21-std', '>25-21', '>25-21-avg',
       '>25-21-std', 'ACC-21

In [ ]:
print("No Injury: " + str(cumulative_df[cumulative_df["Injury"] == 0].shape[0]))
print("Injury: " + str(cumulative_df[cumulative_df["Injury"] == 1].shape[0]))
print(f"%: {((cumulative_df[cumulative_df['Injury'] == 1].shape[0] / cumulative_df.shape[0]) * 100):.2f}%")

No Injury: 4643
Injury: 37
%: 0.79%


In [49]:
# Define columns to consider

# columns_to_consider = ['TD-3', '>19.8-3', '>25-3',
#        'ACC-3', 'DEC-3', 'Sprints-3', 'Mins-3', 'TD-7', '>19.8-7', '>25-7',
#        'ACC-7', 'DEC-7', 'Sprints-7', 'Mins-7', 'TD-21', '>19.8-21', '>25-21',
#        'ACC-21', 'DEC-21', 'Sprints-21', 'Mins-21']

columns_to_consider = ['TD-7', '>19.8-7', '>25-7',
       'ACC-7', 'DEC-7', 'Sprints-7', 'Mins-7', 'TD-21', '>19.8-21', '>25-21',
       'ACC-21', 'DEC-21', 'Sprints-21', 'Mins-21']

# Count zero values in the specified columns for each row
zero_counts = (cumulative_df[columns_to_consider] == 0).sum(axis=1)

# Filter out rows with 3 or more zeros in the specified columns
cumulative_df = cumulative_df[zero_counts < 3]

In [ ]:
# Count of rows containing 0 for each column
zero_counts = (cumulative_df == 0).sum()

# Creating a DataFrame for the results
result_df = pd.DataFrame({
    "Column Name": zero_counts.index,
    "Number of Rows with 0": zero_counts.values
})

result_df

,Column Name,Number of Rows with 0
0,DATE,0
1,Microcycle,0
2,Injury,4381
3,season,0
4,preseason-season,0
...,...,...
81,Sprints-21-avg,28
82,Sprints-21-std,28
83,Mins-21,0
84,Mins-21-avg,0


In [51]:
print("No Injury: " + str(cumulative_df[cumulative_df["Injury"] == 0].shape[0]))
print("Injury: " + str(cumulative_df[cumulative_df["Injury"] == 1].shape[0]))
print(f"%: {((cumulative_df[cumulative_df['Injury'] == 1].shape[0] / cumulative_df.shape[0]) * 100):.2f}%")

No Injury: 4381
Injury: 36
%: 0.82%


## Calculate different load metrics

ACWR and MSWR values, trying to replicate the research paper

In [ ]:
cumulative_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4417 entries, 2 to 4722
Data columns (total 86 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   DATE                4417 non-null   datetime64[ns]
 1   Microcycle          4219 non-null   object        
 2   Injury              4417 non-null   float64       
 3   season              4417 non-null   object        
 4   preseason-season    4417 non-null   object        
 5   TD                  4417 non-null   float64       
 6   >19.8               4417 non-null   float64       
 7   >25                 4417 non-null   float64       
 8   ACC                 4417 non-null   float64       
 9   DEC                 4417 non-null   float64       
 10  PlayerID            4417 non-null   float64       
 11  Player Position     4417 non-null   object        
 12  Max Speed           4417 non-null   float64       
 13  Sprints             4417 non-null   float64       
 1

In [53]:
cumulative_df['ACC-7-std'] = cumulative_df['ACC-7-std'].replace(0, 0.0001)
cumulative_df['DEC-7-std'] = cumulative_df['DEC-7-std'].replace(0, 0.0001)
cumulative_df['>25-7-std'] = cumulative_df['>25-7-std'].replace(0, 0.0001)
cumulative_df.loc[cumulative_df['>25-21-avg'] == 0, '>25-21-avg'] = cumulative_df.groupby('PlayerID')['>25-21-avg'].transform('mean')
cumulative_df.loc[cumulative_df['ACC-21-avg'] == 0, 'ACC-21-avg'] = cumulative_df.groupby('PlayerID')['ACC-21-avg'].transform('mean')
cumulative_df.loc[cumulative_df['DEC-21-avg'] == 0, 'DEC-21-avg'] = cumulative_df.groupby('PlayerID')['DEC-21-avg'].transform('mean')

In [54]:
def calculate_metrics_loads(df, metrics):

  # Calculate ACWR, MSWR for each metric
  for metric in metrics:

      # Calculate 7-day and 28-day averages for ACWR
      df[f'{metric}_ACWR'] = df[f'{metric}-7-avg'] / df[f'{metric}-21-avg']

      # Calculate mean and standard deviation for MSWR
      df[f'{metric}_MSWR'] = df[f'{metric}-7-avg'] / df[f'{metric}-7-std']

  return df

cols_calculate = ['TD', '>19.8', '>25', 'ACC', 'DEC']
cumulative_df = calculate_metrics_loads(cumulative_df, cols_calculate)

cumulative_df.drop(columns=['TD-7-avg', 'TD-7-std',
'>19.8-7-avg', '>19.8-7-std', '>25-7-avg',
       '>25-7-std', 'ACC-7-avg', 'ACC-7-std', 'DEC-7-avg',
       'DEC-7-std', 'Sprints-7-avg', 'Sprints-7-std',
       'Mins-7-avg', 'Mins-7-std', 'TD-21-avg', 'TD-21-std', '>19.8-21-avg', '>19.8-21-std', '>25-21-avg',
       '>25-21-std', 'ACC-21-avg', 'ACC-21-std',
       'DEC-21-avg', 'DEC-21-std', 'Sprints-21-avg',
       'Sprints-21-std', 'Mins-21-avg', 'Mins-21-std'] , inplace=True)

In [55]:
cumulative_df.describe()

,DATE,Injury,TD,>19.8,>25,ACC,DEC,PlayerID,Max Speed,Sprints,Mins,Max Speed Season,Avg Speed Season,% Max Speed,Speed Diff Max Avg,TD_Rel,>19.8_Rel,>25_Rel,ACC_Rel,DEC_Rel,TD-1,>19.8-1,>25-1,ACC-1,DEC-1,Sprints-1,Mins-1,TD_Rel-1,>19.8_Rel-1,>25_Rel-1,ACC_Rel-1,DEC_Rel-1,% Max Speed-1,TD-3,>19.8-3,>25-3,ACC-3,DEC-3,Sprints-3,Mins-3,TD-7,>19.8-7,>25-7,ACC-7,DEC-7,Sprints-7,Mins-7,TD-21,>19.8-21,>25-21,ACC-21,DEC-21,Sprints-21,Mins-21,TD_ACWR,TD_MSWR,>19.8_ACWR,>19.8_MSWR,>25_ACWR,>25_MSWR,ACC_ACWR,ACC_MSWR,DEC_ACWR,DEC_MSWR
count,4417,4417.000000,4417.000000,4417.000000,4417.000000,4417.000000,4417.000000,4417.000000,4417.000000,4417.000000,4417.000000,4417.000000,4417.000000,4417.000000,4417.000000,4417.000000,4417.000000,4417.000000,4417.000000,4417.000000,4417.000000,4417.000000,4417.000000,4417.000000,4417.000000,4417.000000,4417.000000,4417.000000,4417.000000,4417.000000,4417.000000,4417.000000,4417.000000,4417.000000,4417.000000,4417.000000,4417.000000,4417.000000,4417.000000,4417.000000,4417.000000,4417.000000,4417.000000,4417.000000,4417.000000,4417.000000,4417.000000,4417.000000,4417.000000,4417.000000,4417.000000,4417.000000,4417.000000,4417.000000,4417.000000,4417.000000,4417.000000,4417.000000,4417.000000,4417.000000,4417.000000,4417.000000,4417.000000,4417.000000
mean,2022-07-28 15:50:00.135838720,0.008150,4870.039008,189.743604,39.997510,47.980581,39.345076,34894.855332,27.351595,14.132216,66.867082,35.102974,21.452205,77.962589,27.519623,49.438974,34.813799,22.022821,54.636122,43.927007,2784.344849,91.510301,14.301336,29.202270,21.892462,6.802581,42.773407,28.330713,16.993858,8.286672,33.322567,24.625311,50.582642,9133.926851,352.180326,67.652253,91.949701,72.845733,25.678741,128.289100,21171.698777,816.664365,163.144442,211.151301,169.418452,59.630518,294.780319,57441.480009,2215.799638,442.991170,568.397066,459.542395,161.233643,794.355703,1.185781,1.114728,1.204546,0.775372,1.207252,0.553598,1.200461,130.135413,1.199477,73.458031
min,2021-08-12 00:00:00,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000,10103.000000,1.810000,0.000000,4.040000,29.760000,17.940344,5.890010,-91.074358,0.010000,0.000000,0.000000,0.960000,0.930000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1378.000000,1.000000,0.000000,7.000000,1.000000,0.000000,6.350000,2765.000000,2.000000,0.000000,15.000000,1.000000,0.000000,21.510000,0.123969,0.377964,0.001156,0.377964,0.000000,0.000000,0.039848,0.377964,0.014815,0.377964
25%,2022-03-09 00:00:00,0.000000,3164.000000,41.000000,0.000000,30.000000,21.000000,18096.000000,24.510000,3.000000,51.750000,34.380000,20.742552,70.322161,15.209046,31.800000,7.630000,0.000000,35.290000,23.350000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6286.000000,160.000000,7.000000,62.000000,45.000000,9.000000,90.200000,17535.000000,538.000000,65.000000,166.000000,126.000000,33.000000,247.320000,44782.000000,1517.000000,208.000000,423.000000,319.000000,82.000000,605.500000,0.898859,0.868595,0.821014,0.602794,0.700820,0.437559,0.891429,0.860023,0.884311,0.775526
50%,2022-08-04 00:00:00,0.000000,4388.000000,121.000000,9.000000,46.000000,36.000000,26485.000000,27.590000,10.000000,67.230000,35.070000,21.624487,78.956624,28.612675,44.950000,22.290000,5.380000,52.740000,39.530000,2987.000000,32.000000,0.000000,27.000000,18.000000,2.000000,50.200000,29.980000,6.000000,0.000000,32.110000,20.410000,68.776824,9360.000000,326.000000,44.000000,90.000000,73.000000,24.000000,132.700000,21885.000000,787.000000,134.000000,215.000000,176.000000,59.000000,309.930000,61792.000000,2204.000000,377.000000,607.000000,495.000000,164.000000,879.120000,1.037205,1.037354,1.046389,0.717103,1.072115,0.554700,1.041429,1.033630,1.051661,0.957582
75%,2022-12-21 00:00:00,0.000000,5946.000000,290.000000,55.000000,64.000000,53.000000,48692.000000

In [ ]:
cumulative_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4417 entries, 2 to 4722
Data columns (total 68 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   DATE                4417 non-null   datetime64[ns]
 1   Microcycle          4219 non-null   object        
 2   Injury              4417 non-null   float64       
 3   season              4417 non-null   object        
 4   preseason-season    4417 non-null   object        
 5   TD                  4417 non-null   float64       
 6   >19.8               4417 non-null   float64       
 7   >25                 4417 non-null   float64       
 8   ACC                 4417 non-null   float64       
 9   DEC                 4417 non-null   float64       
 10  PlayerID            4417 non-null   float64       
 11  Player Position     4417 non-null   object        
 12  Max Speed           4417 non-null   float64       
 13  Sprints             4417 non-null   float64       
 1

# Do the shift for X previous days injury

If a player got injured X day, it means that the player has been loaded and could have happened the previous session or maybe in the previous 2 sessions.
To add more Injury columns, make the previous 2 days as Injury = 1.

In [57]:
cumulative_df

,DATE,Microcycle,Injury,season,preseason-season,TD,>19.8,>25,ACC,DEC,PlayerID,Player Position,Max Speed,Sprints,Mins,Max Speed Season,Avg Speed Season,% Max Speed,Speed Diff Max Avg,TD_Rel,>19.8_Rel,>25_Rel,ACC_Rel,DEC_Rel,TD-1,>19.8-1,>25-1,ACC-1,DEC-1,Sprints-1,Mins-1,TD_Rel-1,>19.8_Rel-1,>25_Rel-1,ACC_Rel-1,DEC_Rel-1,% Max Speed-1,TD-3,>19.8-3,>25-3,ACC-3,DEC-3,Sprints-3,Mins-3,TD-7,>19.8-7,>25-7,ACC-7,DEC-7,Sprints-7,Mins-7,TD-21,>19.8-21,>25-21,ACC-21,DEC-21,Sprints-21,Mins-21,TD_ACWR,TD_MSWR,>19.8_ACWR,>19.8_MSWR,>25_ACWR,>25_MSWR,ACC_ACWR,ACC_MSWR,DEC_ACWR,DEC_MSWR
2,2021-08-12,M-2,0.0,2021-2022,SEASON,1268.0,0.0,0.0,11.0,5.0,23085.0,CENTRE MIDFIELDER,16.57,0.0,43.58,30.73,20.278653,53.921250,-18.288460,11.71,0.00,0.00,13.59,5.76,7423.0,426.0,20.0,52.0,38.0,24.0,90.85,68.55,73.81,26.13,64.23,43.80,87.178653,13074.0,623.0,54.0,115.0,74.0,43.0,171.20,13074.0,623.0,54.0,115.0,74.0,43.0,171.20,13074.0,623.0,54.0,115.0,74.0,43.0,171.20,1.000000,5.217107,1.000000,1.923701,1.000000,2.727412,1.000000,7.392480,1.000000,26.162951
3,2021-08-13,M-1,0.0,2021-2022,SEASON,4135.0,112.0,2.0,47.0,20.0,23085.0,CENTRE MIDFIELDER,25.47,11.0,58.35,30.73,20.278653,82.883176,25.600056,38.19,19.40,2.61,58.05,23.05,1268.0,0.0,0.0,11.0,5.0,0.0,43.58,11.71,0.00,0.00,13.59,5.76,53.921250,14342.0,623.0,54.0,126.0,79.0,43.0,214.78,14342.0,623.0,54.0,126.0,79.0,43.0,214.78,14342.0,623.0,54.0,126.0,79.0,43.0,214.78,1.000000,1.508831,1.000000,0.974045,1.000000,1.053370,1.000000,1.532602,1.000000,1.423250
4,2021-08-15,M+1,0.0,2021-2022,SEASON,6379.0,86.0,0.0,49.0,48.0,23085.0,CENTRE MIDFIELDER,23.37,13.0,79.48,30.73,20.278653,76.049463,15.244339,58.91,14.90,0.00,60.52,55.33,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,5403.0,112.0,2.0,58.0,25.0,11.0,101.93,18477.0,735.0,56.0,173.0,99.0,54.0,273.13,18477.0,735.0,56.0,173.0,99.0,54.0,273.13,1.000000,1.207543,1.000000,0.832277,1.000000,0.733422,1.000000,1.259807,1.000000,1.140874
5,2021-08-17,M-4,0.0,2021-2022,SEASON,6287.0,209.0,9.0,62.0,44.0,23085.0,CENTRE MIDFIELDER,26.80,19.0,83.69,30.73,20.278653,87.211194,32.158677,58.06,36.21,11.76,76.58,50.72,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,6379.0,86.0,0.0,49.0,48.0,13.0,79.48,24856.0,821.0,56.0,222.0,147.0,67.0,352.61,24856.0,821.0,56.0,222.0,147.0,67.0,352.61,1.000000,1.140383,1.000000,0.757251,1.000000,0.587643,1.000000,1.176025,1.000000,1.053960
6,2021-08-18,M-3,0.0,2021-2022,SEASON,6543.0,166.0,9.0,39.0,35.0,23085.0,CENTRE MIDFIELDER,26.68,13.0,82.64,30.73,20.278653,86.820696,31.566922,60.42,28.76,11.76,48.17,40.35,6287.0,209.0,9.0,62.0,44.0,19.0,83.69,58.06,36.21,11.76,76.58,50.72,87.211194,12666.0,295.0,9.0,111.0,92.0,32.0,163.17,25492.0,833.0,31.0,221.0,155.0,67.0,355.95,31143.0,1030.0,65.0,284.0,191.0,86.0,436.30,0.935482,1.140354,0.924272,0.762941,0.545055,0.581739,0.889336,1.179096,0.927450,1.048327
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4718,2022-05-16,M+1,0.0,2021-2022,SEASON,5436.0,284.0,41.0,69.0,57.0,42579.0,CENTRE BACK,28.31,22.0,71.47,29.76,19.989098,95.127688,41.627200,68.87,76.52,143.23,85.76,78.58,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,6412.0,24.0,0.0,102.0,74.0,4.0,138.71,16067.0,305.0,0.0,190.2,139.8,19.0,248.80,46116.0,1260.0,73.0,550.2,430.8,87.0,718.69,1.045212,1.195538,0.726190,0.569982,0.000000,0.000000,1.037077,1.218145,0.973538,1.144551
4719,2022-05-18,M-4,0.0,2021-2022,SEASON,4514.0,68.0,0.0,60.0,40.0,42579.0,CENTRE BACK,23.36,8.0,85.65,29.76,19.989098,78.494624,16.863701,57.19,18.32,0.00,74.57,55.14,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,5436.0,284.0,41.0,69.0,57.0,22.0,71.47,17271.0,383.0,41.0,214.0,166.0,35.0,281.15,39887.0,937.0,66.0,482.2,381.8,71.0,623.32,1.298995,0.994472,1.226254,0.523207,1.863636,0.

In [58]:
def shift_injuries_x_days(df, x):
    copy_df = df.copy()

    # Iterate over the df to shift injury labels
    for idx, row in copy_df.iterrows():
        if row['Injury'] == 1:
            # Get the player's ID and the injury date
            player_id = row['PlayerID']
            injury_date = row['DATE']

            # Shift injury labels for the current day and the previous x days
            for i in range(0, x + 1):
                prev_date = pd.to_datetime(injury_date) - pd.Timedelta(days=i)
                mask = (copy_df['PlayerID'] == player_id) & (copy_df['DATE'] == prev_date)
                copy_df.loc[mask, 'Injury'] = 1

    return copy_df

In [59]:
# One previous day
# df_injury_shifted = shift_injuries_x_days(cumulative_df, 1)
df_injury_shifted = cumulative_df.copy()

In [60]:
print("No Injury: " + str(df_injury_shifted[df_injury_shifted["Injury"] == 0].shape[0]))
print("Injury: " + str(df_injury_shifted[df_injury_shifted["Injury"] == 1].shape[0]))
print(f"%: {((df_injury_shifted[df_injury_shifted['Injury'] == 1].shape[0] / df_injury_shifted.shape[0]) * 100):.2f}%")

No Injury: 4381
Injury: 36
%: 0.82%


In [61]:
df_injury_shifted.head()

,DATE,Microcycle,Injury,season,preseason-season,TD,>19.8,>25,ACC,DEC,PlayerID,Player Position,Max Speed,Sprints,Mins,Max Speed Season,Avg Speed Season,% Max Speed,Speed Diff Max Avg,TD_Rel,>19.8_Rel,>25_Rel,ACC_Rel,DEC_Rel,TD-1,>19.8-1,>25-1,ACC-1,DEC-1,Sprints-1,Mins-1,TD_Rel-1,>19.8_Rel-1,>25_Rel-1,ACC_Rel-1,DEC_Rel-1,% Max Speed-1,TD-3,>19.8-3,>25-3,ACC-3,DEC-3,Sprints-3,Mins-3,TD-7,>19.8-7,>25-7,ACC-7,DEC-7,Sprints-7,Mins-7,TD-21,>19.8-21,>25-21,ACC-21,DEC-21,Sprints-21,Mins-21,TD_ACWR,TD_MSWR,>19.8_ACWR,>19.8_MSWR,>25_ACWR,>25_MSWR,ACC_ACWR,ACC_MSWR,DEC_ACWR,DEC_MSWR
2,2021-08-12,M-2,0.0,2021-2022,SEASON,1268.0,0.0,0.0,11.0,5.0,23085.0,CENTRE MIDFIELDER,16.57,0.0,43.58,30.73,20.278653,53.921250,-18.288460,11.71,0.00,0.00,13.59,5.76,7423.0,426.0,20.0,52.0,38.0,24.0,90.85,68.55,73.81,26.13,64.23,43.80,87.178653,13074.0,623.0,54.0,115.0,74.0,43.0,171.20,13074.0,623.0,54.0,115.0,74.0,43.0,171.20,13074.0,623.0,54.0,115.0,74.0,43.0,171.20,1.000000,5.217107,1.000000,1.923701,1.000000,2.727412,1.000000,7.392480,1.00000,26.162951
3,2021-08-13,M-1,0.0,2021-2022,SEASON,4135.0,112.0,2.0,47.0,20.0,23085.0,CENTRE MIDFIELDER,25.47,11.0,58.35,30.73,20.278653,82.883176,25.600056,38.19,19.40,2.61,58.05,23.05,1268.0,0.0,0.0,11.0,5.0,0.0,43.58,11.71,0.00,0.00,13.59,5.76,53.921250,14342.0,623.0,54.0,126.0,79.0,43.0,214.78,14342.0,623.0,54.0,126.0,79.0,43.0,214.78,14342.0,623.0,54.0,126.0,79.0,43.0,214.78,1.000000,1.508831,1.000000,0.974045,1.000000,1.053370,1.000000,1.532602,1.00000,1.423250
4,2021-08-15,M+1,0.0,2021-2022,SEASON,6379.0,86.0,0.0,49.0,48.0,23085.0,CENTRE MIDFIELDER,23.37,13.0,79.48,30.73,20.278653,76.049463,15.244339,58.91,14.90,0.00,60.52,55.33,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,5403.0,112.0,2.0,58.0,25.0,11.0,101.93,18477.0,735.0,56.0,173.0,99.0,54.0,273.13,18477.0,735.0,56.0,173.0,99.0,54.0,273.13,1.000000,1.207543,1.000000,0.832277,1.000000,0.733422,1.000000,1.259807,1.00000,1.140874
5,2021-08-17,M-4,0.0,2021-2022,SEASON,6287.0,209.0,9.0,62.0,44.0,23085.0,CENTRE MIDFIELDER,26.80,19.0,83.69,30.73,20.278653,87.211194,32.158677,58.06,36.21,11.76,76.58,50.72,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,6379.0,86.0,0.0,49.0,48.0,13.0,79.48,24856.0,821.0,56.0,222.0,147.0,67.0,352.61,24856.0,821.0,56.0,222.0,147.0,67.0,352.61,1.000000,1.140383,1.000000,0.757251,1.000000,0.587643,1.000000,1.176025,1.00000,1.053960
6,2021-08-18,M-3,0.0,2021-2022,SEASON,6543.0,166.0,9.0,39.0,35.0,23085.0,CENTRE MIDFIELDER,26.68,13.0,82.64,30.73,20.278653,86.820696,31.566922,60.42,28.76,11.76,48.17,40.35,6287.0,209.0,9.0,62.0,44.0,19.0,83.69,58.06,36.21,11.76,76.58,50.72,87.211194,12666.0,295.0,9.0,111.0,92.0,32.0,163.17,25492.0,833.0,31.0,221.0,155.0,67.0,355.95,31143.0,1030.0,65.0,284.0,191.0,86.0,436.30,0.935482,1.140354,0.924272,0.762941,0.545055,0.581739,0.889336,1.179096,0.92745,1.048327


# One Hot Encoding

One Hot Encoding for Player Position

In [62]:
df_injury_shifted['Player Position'] = df_injury_shifted['Player Position'].str.replace(" ", "")
df_injury_shifted['Player Position'].unique()

# Example: One-hot encoding 'Player Position' column
encoded_final_df = pd.get_dummies(df_injury_shifted, columns=['Player Position'], prefix='Position')

columns_to_convert = [
    'Position_CENTREBACK',
    'Position_CENTREMIDFIELDER',
    'Position_FULLBACK',
    'Position_STRIKER',
    'Position_WINGER'
]

# Convert the columns from bool to int
encoded_final_df[columns_to_convert] = encoded_final_df[columns_to_convert].astype(int)

# Display the resulting dataframe
encoded_final_df.head()

,DATE,Microcycle,Injury,season,preseason-season,TD,>19.8,>25,ACC,DEC,PlayerID,Max Speed,Sprints,Mins,Max Speed Season,Avg Speed Season,% Max Speed,Speed Diff Max Avg,TD_Rel,>19.8_Rel,>25_Rel,ACC_Rel,DEC_Rel,TD-1,>19.8-1,>25-1,ACC-1,DEC-1,Sprints-1,Mins-1,TD_Rel-1,>19.8_Rel-1,>25_Rel-1,ACC_Rel-1,DEC_Rel-1,% Max Speed-1,TD-3,>19.8-3,>25-3,ACC-3,DEC-3,Sprints-3,Mins-3,TD-7,>19.8-7,>25-7,ACC-7,DEC-7,Sprints-7,Mins-7,TD-21,>19.8-21,>25-21,ACC-21,DEC-21,Sprints-21,Mins-21,TD_ACWR,TD_MSWR,>19.8_ACWR,>19.8_MSWR,>25_ACWR,>25_MSWR,ACC_ACWR,ACC_MSWR,DEC_ACWR,DEC_MSWR,Position_CENTREBACK,Position_CENTREMIDFIELDER,Position_FULLBACK,Position_STRIKER,Position_WINGER
2,2021-08-12,M-2,0.0,2021-2022,SEASON,1268.0,0.0,0.0,11.0,5.0,23085.0,16.57,0.0,43.58,30.73,20.278653,53.921250,-18.288460,11.71,0.00,0.00,13.59,5.76,7423.0,426.0,20.0,52.0,38.0,24.0,90.85,68.55,73.81,26.13,64.23,43.80,87.178653,13074.0,623.0,54.0,115.0,74.0,43.0,171.20,13074.0,623.0,54.0,115.0,74.0,43.0,171.20,13074.0,623.0,54.0,115.0,74.0,43.0,171.20,1.000000,5.217107,1.000000,1.923701,1.000000,2.727412,1.000000,7.392480,1.00000,26.162951,0,1,0,0,0
3,2021-08-13,M-1,0.0,2021-2022,SEASON,4135.0,112.0,2.0,47.0,20.0,23085.0,25.47,11.0,58.35,30.73,20.278653,82.883176,25.600056,38.19,19.40,2.61,58.05,23.05,1268.0,0.0,0.0,11.0,5.0,0.0,43.58,11.71,0.00,0.00,13.59,5.76,53.921250,14342.0,623.0,54.0,126.0,79.0,43.0,214.78,14342.0,623.0,54.0,126.0,79.0,43.0,214.78,14342.0,623.0,54.0,126.0,79.0,43.0,214.78,1.000000,1.508831,1.000000,0.974045,1.000000,1.053370,1.000000,1.532602,1.00000,1.423250,0,1,0,0,0
4,2021-08-15,M+1,0.0,2021-2022,SEASON,6379.0,86.0,0.0,49.0,48.0,23085.0,23.37,13.0,79.48,30.73,20.278653,76.049463,15.244339,58.91,14.90,0.00,60.52,55.33,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,5403.0,112.0,2.0,58.0,25.0,11.0,101.93,18477.0,735.0,56.0,173.0,99.0,54.0,273.13,18477.0,735.0,56.0,173.0,99.0,54.0,273.13,1.000000,1.207543,1.000000,0.832277,1.000000,0.733422,1.000000,1.259807,1.00000,1.140874,0,1,0,0,0
5,2021-08-17,M-4,0.0,2021-2022,SEASON,6287.0,209.0,9.0,62.0,44.0,23085.0,26.80,19.0,83.69,30.73,20.278653,87.211194,32.158677,58.06,36.21,11.76,76.58,50.72,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,6379.0,86.0,0.0,49.0,48.0,13.0,79.48,24856.0,821.0,56.0,222.0,147.0,67.0,352.61,24856.0,821.0,56.0,222.0,147.0,67.0,352.61,1.000000,1.140383,1.000000,0.757251,1.000000,0.587643,1.000000,1.176025,1.00000,1.053960,0,1,0,0,0
6,2021-08-18,M-3,0.0,2021-2022,SEASON,6543.0,166.0,9.0,39.0,35.0,23085.0,26.68,13.0,82.64,30.73,20.278653,86.820696,31.566922,60.42,28.76,11.76,48.17,40.35,6287.0,209.0,9.0,62.0,44.0,19.0,83.69,58.06,36.21,11.76,76.58,50.72,87.211194,12666.0,295.0,9.0,111.0,92.0,32.0,163.17,25492.0,833.0,31.0,221.0,155.0,67.0,355.95,31143.0,1030.0,65.0,284.0,191.0,86.0,436.30,0.935482,1.140354,0.924272,0.762941,0.545055,0.581739,0.889336,1.179096,0.92745,1.048327,0,1,0,0,0


One Hot Encoding for Match Day

In [63]:
# Encoding logic
# Step 1: Replace ['MD', 'MD(HOME)', 'MD(AWAY)'] with 'MD'
md_values = ['MD', 'MD(HOME)', 'MD(AWAY)']
encoded_final_df['Microcycle'] = encoded_final_df['Microcycle'].replace(md_values, 'MD')

# Step 2: Sort by PlayerID and Date
encoded_final_df.sort_values(by=['PlayerID', 'DATE'], inplace=True)

# Step 3: Shift the Microcycle column by -1 for each PlayerID
encoded_final_df['Microcycle'] = encoded_final_df.groupby('PlayerID')['Microcycle'].shift(1)

# Step 2: Perform one-hot encoding
final_encoded_final_df = pd.get_dummies(encoded_final_df, columns=['Microcycle'], prefix='Microcycle')

one_hot_columns = [col for col in final_encoded_final_df.columns if col.startswith('Microcycle_')]
final_encoded_final_df[one_hot_columns] = final_encoded_final_df[one_hot_columns].astype(int)

final_encoded_final_df = final_encoded_final_df.dropna(subset=one_hot_columns)
# Final encoded DataFrame
print(final_encoded_final_df)

           DATE  Injury     season preseason-season      TD  >19.8    >25  \
2885 2021-08-12     0.0  2021-2022           SEASON  1701.0    0.0    0.0   
2886 2021-08-13     0.0  2021-2022           SEASON  3593.0   54.0    1.0   
2887 2021-08-15     0.0  2021-2022           SEASON  5802.0   92.0    0.0   
2888 2021-08-17     0.0  2021-2022           SEASON  6056.0  275.0   44.0   
2889 2021-08-18     0.0  2021-2022           SEASON  5985.0  288.0  154.0   
...         ...     ...        ...              ...     ...    ...    ...   
556  2023-04-22     0.0  2022-2023           SEASON  4572.0  214.0  101.0   
557  2023-04-24     0.0  2022-2023           SEASON  3788.0  136.0   12.0   
558  2023-04-25     0.0  2022-2023           SEASON  4871.0   83.0    0.0   
560  2023-05-06     0.0  2022-2023           SEASON  2981.0   66.0    0.0   
561  2023-05-09     0.0  2022-2023           SEASON  4444.0   98.0    0.0   

       ACC   DEC  PlayerID  Max Speed  Sprints   Mins  Max Speed Season  \


In [64]:
final_encoded_final_df.head()

,DATE,Injury,season,preseason-season,TD,>19.8,>25,ACC,DEC,PlayerID,Max Speed,Sprints,Mins,Max Speed Season,Avg Speed Season,% Max Speed,Speed Diff Max Avg,TD_Rel,>19.8_Rel,>25_Rel,ACC_Rel,DEC_Rel,TD-1,>19.8-1,>25-1,ACC-1,DEC-1,Sprints-1,Mins-1,TD_Rel-1,>19.8_Rel-1,>25_Rel-1,ACC_Rel-1,DEC_Rel-1,% Max Speed-1,TD-3,>19.8-3,>25-3,ACC-3,DEC-3,Sprints-3,Mins-3,TD-7,>19.8-7,>25-7,ACC-7,DEC-7,Sprints-7,Mins-7,TD-21,>19.8-21,>25-21,ACC-21,DEC-21,Sprints-21,Mins-21,TD_ACWR,TD_MSWR,>19.8_ACWR,>19.8_MSWR,>25_ACWR,>25_MSWR,ACC_ACWR,ACC_MSWR,DEC_ACWR,DEC_MSWR,Position_CENTREBACK,Position_CENTREMIDFIELDER,Position_FULLBACK,Position_STRIKER,Position_WINGER,Microcycle_M+1,Microcycle_M+2,Microcycle_M+3,Microcycle_M-1,Microcycle_M-2,Microcycle_M-3,Microcycle_M-4,Microcycle_M-5,Microcycle_MD
2885,2021-08-12,0.0,2021-2022,SEASON,1701.0,0.0,0.0,14.0,7.0,10103.0,18.04,0.0,71.15,34.92,22.49845,51.660939,-19.816700,16.46,0.00,0.00,12.36,6.73,6534.0,412.0,289.0,53.0,50.0,35.0,90.85,63.22,56.92,129.01,46.79,48.09,96.105384,11655.0,660.0,365.0,139.0,101.0,61.0,171.22,11655.0,660.0,365.0,139.0,101.0,61.0,171.22,11655.0,660.0,365.0,139.0,101.0,61.0,171.22,1.000000,5.832505,1.000000,2.845674,1.00000,1.211709,1.000000,2.978419,1.000000,71.417785,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2886,2021-08-13,0.0,2021-2022,SEASON,3593.0,54.0,1.0,45.0,22.0,10103.0,25.46,7.0,58.35,34.92,22.49845,72.909507,13.163349,34.76,7.46,0.45,39.73,21.16,1701.0,0.0,0.0,14.0,7.0,0.0,71.15,16.46,0.00,0.00,12.36,6.73,51.660939,13356.0,660.0,365.0,153.0,108.0,61.0,242.37,13356.0,660.0,365.0,153.0,108.0,61.0,242.37,13356.0,660.0,365.0,153.0,108.0,61.0,242.37,1.000000,1.791561,1.000000,1.060638,1.00000,0.812116,1.000000,1.415030,1.000000,1.433137,0,0,1,0,0,0,0,0,0,1,0,0,0,0
2887,2021-08-15,0.0,2021-2022,SEASON,5802.0,92.0,0.0,94.0,53.0,10103.0,25.07,14.0,79.48,34.92,22.49845,71.792669,11.429896,56.14,12.71,0.00,82.99,50.98,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,5294.0,54.0,1.0,59.0,29.0,7.0,129.50,16949.0,714.0,366.0,198.0,130.0,68.0,300.72,16949.0,714.0,366.0,198.0,130.0,68.0,300.72,1.000000,1.298328,1.000000,0.785479,1.00000,0.585568,1.000000,1.169623,1.000000,1.095283,0,0,1,0,0,0,0,0,1,0,0,0,0,0
2888,2021-08-17,0.0,2021-2022,SEASON,6056.0,275.0,44.0,74.0,60.0,10103.0,27.83,21.0,83.69,34.92,22.49845,79.696449,23.697408,58.59,37.99,19.64,65.33,57.71,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,5802.0,92.0,0.0,94.0,53.0,14.0,79.48,22751.0,806.0,366.0,292.0,183.0,82.0,380.20,22751.0,806.0,366.0,292.0,183.0,82.0,380.20,1.000000,1.194238,1.000000,0.728658,1.00000,0.483514,1.000000,1.072219,1.000000,1.058659,0,0,1,0,0,1,0,0,0,0,0,0,0,0
2889,2021-08-18,0.0,2021-2022,SEASON,5985.0,288.0,154.0,73.0,79.0,10103.0,34.92,28.0,82.62,34.92,22.49845,100.000000,55.210689,57.91,39.79,68.74,64.45,75.98,6056.0,275.0,44.0,74.0,60.0,21.0,83.69,58.59,37.99,19.64,65.33,57.71,79.696449,11858.0,367.0,44.0,168.0,113.0,35.0,163.17,23686.0,833.0,334.0,280.0,192.0,77.0,383.52,28807.0,1081.0,410.0,366.0,243.0,103.0,463.89,0.939692,1.187842,0.880666,0.734200,0.93101,0.443266,0.874317,1.085851,0.902998,1.039713,0,0,1,0,0,0,0,0,0,0,0,1,0,0


# Export data

In [65]:
#encoded_final_df.to_excel('/content/drive/MyDrive/WATFORD FC/Datos GPS/datos_finales_tommy2.xlsx', index=False)

In [66]:
final_encoded_final_df.to_excel('data/cleaned_data_no_injured.xlsx', index=False)